# Create and Test Vega-Lite UC Function (Simple Version)

This is a simplified version that returns hardcoded Vega-Lite specs.
Use this to test the UC Function creation process without needing Foundation Model access.

## Prerequisites
- USAGE + CREATE permissions on your catalog.schema
- That's it! No LLM access needed.

In [ ]:
%pip install databricks-sdk unitycatalog-ai typing_extensions>=4.7.0 databricks-connect>=15.1.0 --upgrade
dbutils.library.restartPython()  # Comment out for local execution via dbconnect

## Step 1: Configure Your Catalog and Schema

In [ ]:
# Import configuration
%run ../config.py

# Verify you have permissions
spark.sql(f"USE CATALOG {catalog}")
spark.sql(f"USE SCHEMA {schema}")
print(f"✅ Using {catalog}.{schema}")

## Step 2: Define a Simple Vega-Lite Generator

This version returns template specs based on keywords (no LLM needed).

In [ ]:
def generate_vega_lite_spec(
    chart_description: str,
    data_sample: str
) -> str:
    """
    Generate a Vega-Lite visualization specification from a description.
    
    This function is designed to work with LLM agents that can describe
    chart types and provide data in JSON format.
    
    Args:
        chart_description (str): Natural language description of the chart.
            Examples: "bar chart showing sales by region", 
                     "line chart of revenue over time",
                     "scatter plot comparing price vs quality"
        data_sample (str): JSON string array of objects. Each object represents
            a data point. Example: '[{"x": 1, "y": 2}, {"x": 3, "y": 4}]'
            If empty or invalid, uses default sample data.
    
    Returns:
        str: JSON string containing a valid Vega-Lite v5 specification
    
    Raises:
        ValueError: If chart_description is empty
    """
    import json
    
    # Validation
    if not chart_description or not chart_description.strip():
        return json.dumps({
            "error": "chart_description cannot be empty",
            "status": "failed"
        })
    
    # Parse data with fallback
    data_values = []
    try:
        if data_sample and data_sample.strip():
            parsed = json.loads(data_sample)
            if isinstance(parsed, list) and len(parsed) > 0:
                data_values = parsed
    except (json.JSONDecodeError, TypeError):
        pass  # Fall through to default data
    
    # Default sample data if none provided or parsing failed
    if not data_values:
        data_values = [
            {"category": "A", "value": 28},
            {"category": "B", "value": 55},
            {"category": "C", "value": 43}
        ]
    
    # Validate data structure
    if not isinstance(data_values, list) or len(data_values) == 0:
        data_values = [{"category": "A", "value": 28}]
    
    # Get field names safely
    try:
        first_item = data_values[0]
        if not isinstance(first_item, dict) or len(first_item.keys()) < 2:
            # Invalid data structure, use defaults
            data_values = [{"category": "A", "value": 28}]
            first_item = data_values[0]
        
        fields = list(first_item.keys())
        x_field = fields[0]
        y_field = fields[1]
    except (IndexError, KeyError, TypeError):
        # Fallback to safe defaults
        x_field = "category"
        y_field = "value"
    
    # Determine chart type from description (case-insensitive)
    description_lower = chart_description.lower()
    
    # Base configuration
    base_spec = {
        "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
        "description": chart_description,
        "data": {"values": data_values},
        "width": 500,
        "height": 300
    }
    
    # Chart type logic with better pattern matching
    if any(word in description_lower for word in ["bar", "column", "histogram"]):
        mark_type = "bar"
        x_type = "nominal"
        y_type = "quantitative"
    elif any(word in description_lower for word in ["line", "trend", "time series"]):
        mark_type = "line"
        x_type = "ordinal"
        y_type = "quantitative"
    elif any(word in description_lower for word in ["scatter", "point", "correlation"]):
        mark_type = "point"
        x_type = "quantitative"
        y_type = "quantitative"
    elif any(word in description_lower for word in ["area", "stacked"]):
        mark_type = "area"
        x_type = "ordinal"
        y_type = "quantitative"
    elif any(word in description_lower for word in ["pie", "donut"]):
        # Pie charts need different encoding
        return json.dumps({
            **base_spec,
            "mark": {"type": "arc", "tooltip": True},
            "encoding": {
                "theta": {"field": y_field, "type": "quantitative"},
                "color": {"field": x_field, "type": "nominal"}
            }
        })
    else:
        # Default to bar chart
        mark_type = "bar"
        x_type = "nominal"
        y_type = "quantitative"
    
    # Build final spec
    spec = {
        **base_spec,
        "mark": {
            "type": mark_type,
            "tooltip": True,
            "point": mark_type == "line"  # Add points to line charts
        },
        "encoding": {
            "x": {
                "field": x_field,
                "type": x_type,
                **({"sort": None} if x_type == "ordinal" else {}),  # Disable sorting for ordinal to preserve data order
                **({"axis": {"labelAngle": 0}} if x_type == "nominal" else {})
            },
            "y": {
                "field": y_field,
                "type": y_type
            }
        },
        "config": {
            "view": {"stroke": None}
        }
    }
    
    return json.dumps(spec)


# Test the function
print("Testing function locally...")
test_result = generate_vega_lite_spec("bar chart with 3 categories", "")  # Empty string for default data
print("✅ Function works!")
print(f"Generated spec:\n{test_result}")

## Step 3: Register as UC Function

In [ ]:
from unitycatalog.ai.core.databricks import DatabricksFunctionClient
from databricks.sdk import WorkspaceClient

# Initialize clients
client = DatabricksFunctionClient()
w = WorkspaceClient()  # For getting workspace URL

full_function_name = f"{catalog}.{schema}.generate_vega_lite_spec"

print(f"Creating UC Function: {full_function_name}")

# Create the function using the high-level API
function_info = client.create_python_function(
    func=generate_vega_lite_spec,
    catalog=catalog,
    schema=schema,
    replace=True  # Overwrites existing function if it exists
)

print(f"✅ UC Function created successfully!")
print(f"Function name: {function_info.full_name}")
print(f"\n📍 MCP URL: {w.config.host}/api/2.0/mcp/functions/{catalog}/{schema}")

## Step 4: Test the UC Function

In [ ]:
import json

# Reuse the DatabricksFunctionClient for testing
# client = DatabricksFunctionClient()  # Already initialized above

print("Test 1: Bar chart with default data")
print("-" * 60)
result_1 = client.execute_function(
    full_function_name,
    {"chart_description": "bar chart of sales", "data_sample": ""}  # Empty string for default data
)
print(f"✅ Result:\n{json.dumps(json.loads(result_1.value), indent=2)}")
print()

print("Test 2: Line chart with custom data")
print("-" * 60)
sample_data = json.dumps([
    {"month": "Jan", "revenue": 10000},
    {"month": "Feb", "revenue": 15000},
    {"month": "Mar", "revenue": 13000},
    {"month": "Apr", "revenue": 18000}
])
result_2 = client.execute_function(
    full_function_name,
    {"chart_description": "line chart showing revenue trend", "data_sample": sample_data}
)
print(f"✅ Result:\n{json.dumps(json.loads(result_2.value), indent=2)}")

## Step 5: Visualize the Result

In [ ]:
# Visualize the line chart
vega_spec = json.loads(result_2.value)

# displayHTML only works in Databricks notebooks, not local execution
try:
    displayHTML(f"""
<!DOCTYPE html>
<html>
<head>
  <script src="https://cdn.jsdelivr.net/npm/vega@5"></script>
  <script src="https://cdn.jsdelivr.net/npm/vega-lite@5"></script>
  <script src="https://cdn.jsdelivr.net/npm/vega-embed@6"></script>
  <style>
    body {{ background: #f5f5f5; padding: 20px; }}
    #vis {{ background: white; padding: 20px; border-radius: 8px; }}
  </style>
</head>
<body>
  <h2>Generated Vega-Lite Visualization</h2>
  <div id="vis"></div>
  <script>
    vegaEmbed('#vis', {json.dumps(vega_spec)}, {{theme: 'latimes'}});
  </script>
</body>
</html>
""")
except NameError:
    print("Note: displayHTML only works in Databricks notebooks, not local execution")
    print(f"Vega spec generated:\n{json.dumps(vega_spec, indent=2)}")

## Step 6: Test in Serverless Mode (Production)

In [ ]:
print("Testing scatter plot...")
print("-" * 60)
result_prod = client.execute_function(
    full_function_name,
    {
        "chart_description": "scatter plot",
        "data_sample": json.dumps([
            {"x": 1, "y": 2},
            {"x": 2, "y": 4},
            {"x": 3, "y": 6},
            {"x": 4, "y": 8}
        ])
    }
)
print(f"✅ Execution successful!")
print(f"Result: {result_prod.value[:100]}...")

## ✅ Success! Your UC Function is Ready

### What You Created:
- ✅ UC Function: `main.brandon_cowen.generate_vega_lite_spec`
- ✅ Tested in local mode (fast)
- ✅ Tested in serverless mode (production)
- ✅ Visualized the output

### MCP Access:
```
{workspace_url}/api/2.0/mcp/functions/main/brandon_cowen
```

### Next Steps:
1. Configure your agent to use this MCP endpoint
2. The agent will auto-discover `generate_vega_lite_spec`
3. When users ask for visualizations, the agent calls your function
4. Your app renders the returned Vega-Lite spec

### Grant Permissions:
```sql
GRANT EXECUTE ON FUNCTION main.brandon_cowen.generate_vega_lite_spec TO `<principal>`;
```